In [27]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

path = r'/Users/dewiindahpurnama_/Documents/Python/property-data/Sellers_Accounts_2026_01_05.xlsx'
df = pd.read_excel(path, dtype=str, sheet_name='Sheet0', usecols=['Record Id', 'Zip Code', 'Seller Account Owner.id', 'Seller Account Name', 'Phone','Agents','Link'])



/Users/dewiindahpurnama_/Documents/Python/property/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [28]:
df.loc[df['Link'].notna()]

,Record Id,Seller Account Name,Phone,Seller Account Owner.id,Zip Code,Agents,Link
2,zcrm_6965175000003577206,Victor Remy,NaN,zcrm_6965175000000667009,SC1038,Suci,https://matapropertybali.com/property/modern-4...
3,zcrm_6965175000003577207,Cedric Henry Levi,NaN,zcrm_6965175000000667009,SC1017,Suci,https://matapropertybali.com/property/modern-3...
4,zcrm_6965175000003577208,BSH,NaN,zcrm_6965175000000667009,SC1141,Suci,https://matapropertybali.com/property/villa-wi...
5,zcrm_6965175000003577209,Bali Tropical Sharing,NaN,zcrm_6965175000000667009,SC292,Suci,https://matapropertybali.com/property/5-bedroo...
6,zcrm_6965175000003577210,BSH,NaN,zcrm_6965175000000667009,SC194,Suci,https://matapropertybali.com/property/luxuriou...
...,...,...,...,...,...,...,...
527,zcrm_6965175000003616239,CP: Juli Optic,+62 831-1616-7305,zcrm_6965175000000667009,SC554,Suci,https://matapropertybali.com/property/a-breath...
528,zcrm_6965175000003616240,Balix Development,NaN,zcrm_6965175000000667009,SC506,Suci,https://matapropertybali.com/property/stylish-...
529,zcrm_6965175000003616241,CP: Lanny,+62 813-1920-0900,zcrm_6965175000000667009,SC457,Suci,https://matapropertybali.com/property/discover...
530,zcrm_6965175000003616242,JSP Development,+62 812-3661-9575,zcrm_6965175000000667009,SC450,Suci,https://matapropertybali.com/property/newly-bu...


In [29]:
def get_property_data(url):
    if pd.isna(url):
        return {"Scraped_Title": "", "Scraped_Price": ""}

    soup = BeautifulSoup(requests.get(url, timeout=10).text, "html.parser")

    title_tag = soup.select_one(".page-title h1")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # If title is empty → force price empty
    if not title:
        return {"Scraped_Title": "", "Scraped_Price": ""}

    price_tag = soup.select_one("li.item-price")
    price = price_tag.get_text(strip=True) if price_tag else ""

    return {
        "Scraped_Title": title,
        "Scraped_Price": price,
    }


In [30]:
# Apply the function using lambda and expand the dictionary into new columns
df[['Property Name', 'Scraped_Price']] = df['Link'].apply(
    lambda x: pd.Series(get_property_data(x))
).rename(columns={'Scraped_Title': 'Property Name'})

In [31]:
column_order = [
    'Record Id', 
    'Zip Code', 
    'Property Name', 
    'Seller Account Owner.id',
    'Seller Account Name', 
    'Phone', 
    'Agents', 
    'Link', 
    'Scraped_Price'
]

In [35]:
# 3. Rearrange the DataFrame
df = df[column_order]
df

,Record Id,Zip Code,Property Name,Seller Account Owner.id,Seller Account Name,Phone,Agents,Link,Scraped_Price
0,zcrm_6965175000001846208,NaN,,zcrm_6965175000000598187,Rick Leenheer,NaN,NaN,NaN,
1,zcrm_6965175000002165378,NaN,,zcrm_6965175000000598187,Coco,+6281011094,NaN,NaN,
2,zcrm_6965175000003577206,SC1038,Modern 4-Bedroom Sanctuary in Pejeng’s Cultura...,zcrm_6965175000000667009,Victor Remy,NaN,Suci,https://matapropertybali.com/property/modern-4...,"9,380,000,000 IDR"
3,zcrm_6965175000003577207,SC1017,"Modern 3-Bedroom Villa in Singakerta for Sale,...",zcrm_6965175000000667009,Cedric Henry Levi,NaN,Suci,https://matapropertybali.com/property/modern-3...,"5,860,000,000 IDR"
4,zcrm_6965175000003577208,SC1141,"Serene 3-Bedroom Jungle Villa in Tegalalang, Ubud",zcrm_6965175000000667009,BSH,NaN,Suci,https://matapropertybali.com/property/villa-wi...,"6,360,000,000 IDR"
...,...,...,...,...,...,...,...,...,...
527,zcrm_6965175000003616239,SC554,A breathtaking plot of land in Tegalalang for ...,zcrm_6965175000000667009,CP: Juli Optic,+62 831-1616-7305,Suci,https://matapropertybali.com/property/a-breath...,"5,000,000 IDR/are /year"
528,zcrm_6965175000003616240,SC506,"Stylish Apartments in Ubud for Sale, Bali",zcrm_6965175000000667009,Balix Development,NaN,Suci,https://matapropertybali.com/property/stylish-...,"2,650,000,000 IDR"
529,zcrm_6965175000003616241,SC457,Stunning 3-Bedroom Villa with Ocean View in Ce...,zcrm_6965175000000667009,CP: Lanny,+62 813-1920-0900,Suci,https://matapropertybali.com/property/discover...,"5,500,000,000 IDR"
530,zcrm_6965175000003616242,SC450,Newly Built 2 bedroom Villa in Nusadua,zcrm_6965175000000667009,JSP Development,+62 812-3661-9575,Suci,https://matapropertybali.com/property/newly-bu...,"2,700,000,000 IDR"


In [36]:
df.to_excel(r'/Users/dewiindahpurnama_/Documents/Python/property-data/Property Listing Data.xlsx', index=False)